In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"ahmedelkhamisi","key":"f9f4c8dfa48d84d6567952071d66901a"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d rupakroy/online-payments-fraud-detection-dataset

online-payments-fraud-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from zipfile import ZipFile
file_name="online-payments-fraud-detection-dataset.zip" 
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('done')

done


In [71]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import plotly.express as px
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.model_selection import learning_curve
import warnings
warnings.filterwarnings('ignore')

# Read Dataset

In [72]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [73]:
df.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1,0
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1,0


In [74]:
df.shape

(6362620, 11)

In [75]:
df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [76]:
print(df.type.value_counts())

CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: type, dtype: int64


In [77]:
#Sampling on all the types with equal samples for fair training
df_1=df[df["type"]=='PAYMENT']
df_2=df[df["type"]=='CASH_IN']
df_3=df[df["type"]=='TRANSFER']
df_4=df[df["type"]=='DEBIT']
df_0=(df[df["type"]=='CASH_OUT']).sample(n=20000,random_state=42)
df_1_upsample=resample(df_1,replace=True,n_samples=20000,random_state=123)
df_2_upsample=resample(df_2,replace=True,n_samples=20000,random_state=124)
df_3_upsample=resample(df_3,replace=True,n_samples=20000,random_state=125)
df_4_upsample=resample(df_4,replace=True,n_samples=20000,random_state=126)
train_df=pd.concat([df_0,df_1_upsample,df_2_upsample,df_3_upsample,df_4_upsample])

In [78]:
# randomly sampling from each class
classes=train_df.groupby("type",group_keys=False).apply(lambda df : df.sample(1))
classes

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1789211,162,CASH_IN,52238.11,C1682896784,7816642.16,7868880.28,C356500145,9547831.40,9495593.29,0,0
2961215,230,CASH_OUT,25081.38,C1183218339,0.00,0.00,C1447391514,1393528.51,1418609.89,0,0
442507,19,DEBIT,4796.97,C1117976788,7911.00,3114.03,C1758099520,1549866.10,1554663.07,0,0
5882363,403,PAYMENT,34187.47,C1236340485,0.00,0.00,M473321453,0.00,0.00,0,0
5264356,372,TRANSFER,929552.67,C1490739510,0.00,0.00,C385730010,1934875.70,2864428.37,0,0


In [79]:

df.head()
X=df.drop('isFraud', axis=1)
type_new = pd.get_dummies(train_df['type'],drop_first=True)
data = pd.concat([train_df,type_new],axis=1)
data.head()
data_X = data[['step','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud','CASH_OUT','DEBIT','PAYMENT','TRANSFER']]
data_y = data['isFraud']
scaler = preprocessing.MinMaxScaler().fit(data_X)
scaled_feature = scaler.transform(data_X)
data_scaled = pd.DataFrame(scaled_feature,columns= ['step','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','isFlaggedFraud','CASH_OUT','DEBIT','PAYMENT','TRANSFER'])
data_scaled.head()


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,0.405954,0.000903,0.000000,0.000000,0.000460,0.000608,0.0,1.0,0.0,0.0,0.0
1,0.336942,0.001528,0.000016,0.000000,0.012682,0.012932,0.0,1.0,0.0,0.0,0.0
2,0.189445,0.000215,0.000867,0.000805,0.000000,0.000035,0.0,1.0,0.0,0.0,0.0
3,0.705007,0.001688,0.001446,0.000000,0.029385,0.029661,0.0,1.0,0.0,0.0,0.0
4,0.178620,0.002905,0.000000,0.000000,0.005954,0.006430,0.0,1.0,0.0,0.0,0.0


In [80]:
Y=df['isFraud']
X, X_test, y, y_test =train_test_split(data_scaled,
                                      data_y,
                                      test_size=0.2,
                                      random_state=0)
X_train, X_CV, y_train, y_CV =train_test_split(X,y,
                                               test_size=0.25,
                                               random_state=0)
#X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.4 , random_state=1)
#X_train, X_cv, y_train, y_cv = train_test_split(X, Y, test_size=0.25 , random_state=1)

In [81]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 11)
(20000, 11)
(60000,)
(20000,)


In [82]:
LogReg = LogisticRegression()
LogReg.fit(X_train, y_train)

LogisticRegression()

In [83]:
y_pred = LogReg.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [84]:
mse = mean_squared_error(y_test,y_pred)
acc=LogReg.score(X_test,y_test)
#val_p=LogReg.predict(X_cv)
#v_acc=LogReg.score(X_cv,y_cv)

In [85]:
#val_p


In [86]:
print("Mean Squared Error",mse)
print("Accuracy",acc)
#print("Validation Accuracy",v_acc)

Mean Squared Error 0.0022
Accuracy 0.9978


In [87]:
poly_features = PolynomialFeatures(degree=3)
X_2nd= poly_features.fit_transform(X)